In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

In [2]:
df = pd.read_json("280_spam_dropped.json")
df.head()

,comment,category,channel,video_url,video_id,channel_size,identity,score,results
0,8 months? Is that all.,Automobiles,https://www.youtube.com/@autofanaticcars,https://www.youtube.com/watch?v=Wqcy9do6YO0,Wqcy9do6YO0,0,1,1,0
1,impressed very effective,Automobiles,https://www.youtube.com/@autofanaticcars,https://www.youtube.com/watch?v=Wqcy9do6YO0,Wqcy9do6YO0,0,1,2,0
2,"No matter what, you have to use brushes and ag...",Automobiles,https://www.youtube.com/@autofanaticcars,https://www.youtube.com/watch?v=Wqcy9do6YO0,Wqcy9do6YO0,0,1,1,0
3,"Good video BUT, why don't you ""articulate"" tha...",Automobiles,https://www.youtube.com/@autofanaticcars,https://www.youtube.com/watch?v=Wqcy9do6YO0,Wqcy9do6YO0,0,1,1,0
4,"My wheels werenever done until 215k km, they l...",Automobiles,https://www.youtube.com/@autofanaticcars,https://www.youtube.com/watch?v=Wqcy9do6YO0,Wqcy9do6YO0,0,1,2,0


In [3]:
(df["score"] == 2).sum()

30616

In [4]:
tokenizer = Tokenizer(num_words=50, lower=True)
tokenizer.fit_on_texts(df["comment"])
sequences = tokenizer.texts_to_sequences(df["comment"])
data = pad_sequences(sequences, maxlen=300)

In [11]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(50, 100, input_length=300))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
labels = pd.to_numeric(df["score"])
labels

0         1.0
1         2.0
2         1.0
3         1.0
4         2.0
         ... 
189757    0.0
189758    0.0
189759    1.0
189760    2.0
189761    2.0
Name: score, Length: 189762, dtype: float64

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=32)


Epoch 1/5
 400/4745 [=>............................] - ETA: 34:08 - loss: nan - accuracy: 0.1030

KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)